In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pdb
from copy import deepcopy
import os
%matplotlib inline

In [ ]:
def calc_params_calibration(img, num_x_y, draw_points=True):
    nx = num_x_y[0]
    ny = num_x_y[1]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret:
        if draw_points:
            cv2.drawChessboardCorners(raw_cam, (nx, ny), corners, ret)
        objpoints = []
        imgpoints = []
        imgpoints.append(corners)
        objp = np.zeros((nx*ny, 3), np.float32)  # coordinates in 3d (x, y, z)
        # objp[:,:2] x, y coordinates of the points in 3d
        objp[:,:2]=np.mgrid[0:nx,0:ny].T.reshape(-1,2)
        objpoints.append(objp)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        return ret, mtx, dist, corners
    else:
        return False, None, None, None
if not os.path.exists('pics'):
    os.makedirs('pics')
def plot_2pics(img1, img2, titles,  cmap=[None, None], fontsize=15):
    f, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5), dpi=80)
    f.tight_layout()
    ax1.imshow(img1,cmap=cmap[0])
    ax1.axis('off')
    ax1.set_title(titles[0], fontsize=fontsize)
    ax2.imshow(img2,cmap=cmap[1])
    ax2.axis('off')
    ax2.set_title(titles[1], fontsize=fontsize)
    return f

In [ ]:
tst_img_name = './test_images/test1.jpg'
raw_cam = mpimg.imread('./camera_cal/calibration2.jpg')
# mtx, dist as global vars
ret, mtx, dist, corners = calc_params_calibration(raw_cam, (9, 6), False)
undist_cam = cv2.undistort(raw_cam, mtx, dist, None, mtx)
f = plot_2pics(raw_cam, undist_cam, ['Original Image','Undistorted Image'])
plt.savefig('pics/calibration1.png')

In [ ]:
# raw_img = mpimg.imread('./test_images/straight_lines1.jpg')
raw_img = mpimg.imread(tst_img_name)
undist_img = cv2.undistort(raw_img, mtx, dist, None, mtx)
f=plot_2pics(raw_img, undist_img,['Original Image', 'Undistorted Image'])
plt.savefig('pics/calibration2.png')

In [ ]:
def convert_hls(img, thresh_s=(170, 255), thresh_l_der=(20, 100)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_ch = hls[:,:,2]
    l_ch = hls[:,:,1]
    bi_s = np.zeros_like(s_ch)
    bi_s[(s_ch>thresh_s[0])&(s_ch<=thresh_s[1])] = 1
    l_der = cv2.Sobel(l_ch, cv2.CV_64F, 1, 0)
    l_der = np.absolute(l_der)
    l_der = np.uint8(255*l_der/np.max(l_der))
    
    bi_l_der = np.zeros_like(l_der)
    bi_l_der[(l_der>thresh_l_der[0])&(l_der<=thresh_l_der[1])] = 1
    return bi_s, bi_l_der

def mix_2_chs(ch1, ch2, color_img=True):
    mix = np.zeros_like(ch1)
    mix[(ch1==1)|(ch2==1)] = 1
    if color_img:
        color = np.dstack((np.zeros_like(ch1), ch1, ch2))
        color *= 255
        return color, mix
    else:
        return mix

In [ ]:
bi_s, bi_l_der = convert_hls(undist_img)
color, mix = mix_2_chs(bi_s, bi_l_der)
f=plot_2pics(color, mix,['Mixed Image (Colored)', 'Mixed Binary Image'],[None,'gray'])
plt.savefig('pics/imgProcessing.png')

In [ ]:
def point_lin(p0, p1, rate):
    x = int(p0[0] + (p1[0] - p0[0]) * rate)
    y = int(p0[1] + (p1[1] - p0[1]) * rate)
    return [x, y]

def gen_vertices(img, vanish_p=None, rate=0.8):
    offset = 30
    if vanish_p is None:
        cent_p = (img.shape[1]//2, img.shape[0]//2)
    else:
        cent_p = vanish_p
    p0 = [0, img.shape[0]-offset]
    p1 = point_lin(p0, cent_p, rate)
    p3 = [img.shape[1], img.shape[0]-offset]
    p2 = point_lin(p3, cent_p, rate)
    return np.array([p0, p1, p2, p3])

def draw_lines(img, vertices, color=(0,0,255), thickness=5):
    for i in range(len(vertices)-1):
        cv2.line(img, (vertices[i][0],vertices[i][1]), (vertices[i+1][0], vertices[i+1][1]), color, thickness) 

def draw_closed_lines(img, vertices, color=(0,0,255), thickness=5):
    draw_lines(img, vertices, color, thickness)
    cv2.line(img, (vertices[-1][0],vertices[-1][1]), (vertices[0][0], vertices[0][1]), color, thickness)

def warp(img, src):
    size = (img.shape[1], img.shape[0])
    src = np.float32(src)
    dst = np.float32([[0,size[1]],
                      [0,0],
                      [size[0],0],
                      [size[0],size[1]]])
    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(img, M, size, flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [ ]:
vanish_p = [mix.shape[1]//2, int(mix.shape[0]/100*58.3)]  # with several experiments the vanish point will be tuned
vertices = gen_vertices(mix, vanish_p, 0.83)
# rate should be tuned
# rate cannot be too close to 1, because the top of the picture will be more blurred
img_poly = deepcopy(mix)*255
img_poly = np.dstack((img_poly, img_poly, img_poly))
# pdb.set_trace()
draw_closed_lines(img_poly, vertices, (255,0,0))
img_warped, M, Minv = warp(mix, vertices)
f=plot_2pics(img_poly, img_warped,['Undistorted Image','Warped Image'], [None, 'gray'])  
plt.savefig('pics/imgWarping.png')
# the lines in warped picture should be as parallel as possible

In [ ]:
def find_win_centroids(img, win_w, win_h, margin, nonzero, min_pix=0):
    # introduce a minimum pixels number, so that when enough pixel detected then change the center position
    # to avoid that when in this current window, 
    # there are very few pixels, so that the windows's position can be shifted uncertainly
    nonzero_x = nonzero[0]
    nonzero_y = nonzero[1]
    
    l_idx = []
    r_idx = []
    pos_first = 0.5
    win_centroids = []
    offset = win_w/2
    win = np.ones(win_w)
    l_sum = np.sum(img[int(img.shape[0]*pos_first):,:int(img.shape[1]/2)], axis=0)
    l_cent = np.argmax(np.convolve(win, l_sum))-offset
    
    r_sum = np.sum(img[int(img.shape[0]*pos_first):,int(img.shape[1]/2):], axis=0)
    r_cent = np.argmax(np.convolve(win, r_sum))-offset+int(img.shape[1]/2)
    
    # win_centroids.append((l_cent, r_cent))
    
    for i in range(0, int(img.shape[0]/win_h)):
        win_low = int(img.shape[0]-i*win_h)  # bigger than win_high
        win_high = int(img.shape[0]-(i+1)*win_h)  # smaller than win_low
        
        img_layer = np.sum(img[win_high:win_low, :], axis=0)
        # conv_res = np.convolve(win, img_layer)
        
        l_min = int(max(l_cent-margin,0))
        l_max = int(min(l_cent+margin,img.shape[1]))
        l_conv_res = np.convolve(win, img_layer[l_min:l_max])
        if np.max(l_conv_res)>min_pix:  
            l_cent = np.argmax(l_conv_res)-offset+l_min
        
        r_min = int(max(r_cent-margin,0))
        r_max = int(min(r_cent+margin,img.shape[1]))
        r_conv_res = np.convolve(win, img_layer[r_min:r_max])
        if np.max(r_conv_res)>min_pix:
            r_cent = np.argmax(r_conv_res)-offset+r_min
        # r_cent = np.argmax(np.convolve(win, img_layer[r_min:r_max]))-offset+r_min
        
        win_centroids.append((l_cent, r_cent))
        
        win_l_min = l_cent-offset
        win_l_max = l_cent+offset
        
        win_r_min = r_cent - offset
        win_r_max = r_cent + offset
        
        cur_l_idx = ((nonzero_x >= win_high)&(nonzero_x< win_low)&(nonzero_y>=win_l_min)&(nonzero_y<win_l_max)).nonzero()[0]
        cur_r_idx = ((nonzero_x >= win_high)&(nonzero_x< win_low)&(nonzero_y>=win_r_min)&(nonzero_y<win_r_max)).nonzero()[0]
        l_idx.append(cur_l_idx)
        r_idx.append(cur_r_idx)
        # pdb.set_trace()
    
    # pdb.set_trace()
    return win_centroids, (np.concatenate(l_idx), np.concatenate(r_idx))
                           
def win_msk(w, h, img_ref, cent, i):
    out = np.zeros_like(img_ref)
    out[int(img_ref.shape[0]-(i+1)*h):int(img_ref.shape[0]-i*h),max(0,int(cent-w/2)):min(int(cent+w/2),img_ref.shape[1])]=1
    return out

In [ ]:
def get_img_out(bi_img, data_poly):
    plot_y = data_poly[0]
    l_poly_x = data_poly[1]
    r_poly_x = data_poly[2]
    
    out = np.zeros_like(bi_img).astype(np.uint8)
    out = np.dstack((out,out,out))

    # pts_l = np.array([np.transpose(np.vstack([l_poly_x, plot_y]))])
    # pts_r = np.array([np.flipud(np.transpose(np.vstack([l_poly_x, plot_y])))])
    # pts = np.hstack((pts_l, pts_r))

    # pdb.set_trace()
    l_poly_pts = np.int_([l_poly_x, plot_y]).T
    r_poly_pts = np.int_([r_poly_x, plot_y]).T
    pts = np.vstack((l_poly_pts, np.flipud(r_poly_pts)))
    mid_pts = np.int_([(l_poly_x + r_poly_x)/2, plot_y]).T

    cv2.fillPoly(out, np.int_([pts]), (0,255,0))
    # out[np.concatenate([l_y,r_y]), np.concatenate([l_x, r_x])] = [255, 0, 0]

    draw_lines(out, l_poly_pts, (255,0,0), 30)
    draw_lines(out, r_poly_pts, (255,0,0), 30)

    draw_lines(out, mid_pts, (0,0,255), 10)
    return out

def calc_poly_x(y, fit):
    return fit[0]*(y**2)+fit[1]*y+fit[2]

def get_x_y_pts(nonzero, idx_all):
    l_y = nonzero[0][idx_all[0]]  
    l_x = nonzero[1][idx_all[0]]
    r_y = nonzero[0][idx_all[1]]
    r_x = nonzero[1][idx_all[1]]
    return l_y, l_x, r_y, r_x

def find_polynom(bi_img, win_w, win_h, margin, exp_extra=False, min_pix=20):
    nonzero = bi_img.nonzero()
    win_centroids, idx_all = find_win_centroids(bi_img, win_w, win_h, margin, nonzero, min_pix)
    l_y, l_x, r_y, r_x = get_x_y_pts(nonzero, idx_all)
    
    l_fit = np.polyfit(l_y, l_x, 2)
    r_fit = np.polyfit(r_y, r_x, 2)
    
    plot_y = np.linspace(0, mix.shape[0]-1,mix.shape[0])  # for all y
#     l_poly_x = l_fit[0]*plot_y**2+l_fit[1]*plot_y+l_fit[2]
#     r_poly_x = r_fit[0]*plot_y**2+r_fit[1]*plot_y+r_fit[2]
    l_poly_x = calc_poly_x(plot_y, l_fit)
    r_poly_x = calc_poly_x(plot_y, r_fit)
    
    if exp_extra:
        return np.array([plot_y, l_poly_x, r_poly_x]), (win_centroids, idx_all)
    else:
        return np.array([plot_y, l_poly_x, r_poly_x])

def draw_win(bi_img, win_w, win_h, extra, data_poly):
    plot_y = data_poly[0]
    l_poly_x = data_poly[1]
    r_poly_x = data_poly[2]
    nonzero = bi_img.nonzero()
    win_centroids = extra[0]
    idx_all = extra[1]
    
    l_pts_win = np.zeros_like(bi_img)
    r_pts_win = np.zeros_like(bi_img)
    for i in range(0, len(win_centroids)):
        l_msk = win_msk(win_w,win_h,bi_img,win_centroids[i][0],i)
        r_msk = win_msk(win_w,win_h,bi_img,win_centroids[i][1],i)
        l_pts_win[(l_pts_win==255)|(l_msk==1)]=255
        r_pts_win[(r_pts_win==255)|(r_msk==1)]=255
        
    tmplt = np.array(r_pts_win+l_pts_win,np.uint8)
    zero_ch = np.zeros_like(tmplt)
    tmplt = np.dstack((zero_ch, tmplt, zero_ch))
    
    mix_3 = np.dstack((bi_img,bi_img,bi_img))*255
    # mix_3[l_y, l_x] = [255, 0, 0]  # left points
    # mix_3[r_y, r_x] = [0,0,255]  # right points
    
    out = cv2.addWeighted(mix_3, 1, tmplt, 0.5, 0.0)
    l_y, l_x, r_y, r_x = get_x_y_pts(nonzero, idx_all)
    out[l_y, l_x] = [255, 0, 0]  # left points
    out[r_y, r_x] = [0, 0, 255]  # right points
    
    l_poly_pts = np.int_([l_poly_x, plot_y]).T
    r_poly_pts = np.int_([r_poly_x, plot_y]).T
    draw_lines(out, l_poly_pts, (255,255,0))
    draw_lines(out, r_poly_pts, (255,255,0))
    return out

In [ ]:
win_w = 53  # tuned, should fit the with of the line
win_h = 80  # for straight line 180
margin = 100  # should not be too small
min_pix = 20

data_poly, extra = find_polynom(img_warped, win_w, win_h, margin, True)
out_win = draw_win(img_warped, win_w, win_h, extra, data_poly)
out_img = get_img_out(img_warped, data_poly)

f=plot_2pics(out_win, out_img,['Lines Detection through Windows','Lines Detection Output-Image'])
plt.savefig('pics/linesFinding1.png')

In [ ]:
def calc_rad(y, A, B):
    return ((1+(2*A*y+B)**2)**1.5) / np.absolute(2*A)

def get_rad_m(data_poly, m_p_pix_y = 30/720, m_p_pix_x = 3.7/700):
    plot_y = data_poly[0]
    l_poly_x = data_poly[1]
    r_poly_x = data_poly[2]
    
    mid_x = np.int_((l_poly_x+r_poly_x)/2)
    y_eval = np.max(plot_y) * m_p_pix_y
    mid_fit_m = np.polyfit(plot_y*m_p_pix_y, mid_x*m_p_pix_x, 2)
    offset = 1280//2 - mid_x[-1]  # 1280 is the width of the img
    offset *= m_p_pix_x
#     l_fit_m = np.polyfit(plot_y*m_p_pix_y, l_poly_x*m_p_pix_x, 2)
#     r_fit_m = np.polyfit(plot_y*m_p_pix_y, r_poly_x*m_p_pix_x, 2) 
    if mid_fit_m[0] == 0:
        rad = None
    elif mid_fit_m[0] > 0:
        rad = calc_rad(y_eval, mid_fit_m[0], mid_fit_m[1])
    elif mid_fit_m[0] < 0:
        rad = -calc_rad(y_eval, mid_fit_m[0], mid_fit_m[1])
    return rad, offset
#     if l_fit_m[0] == 0:
#         l_rad = None
#     elif l_fit_m[0] > 0:
#         l_rad = calc_rad(y_eval, l_fit_m[0], l_fit_m[1])
#     elif l_fit_m[0] < 0:
#         l_rad = -calc_rad(y_eval, l_fit_m[0], l_fit_m[1])
    
#     if r_fit_m[0] == 0:
#         r_rad = None
#     elif r_fit_m[0] > 0:
#         r_rad = calc_rad(y_eval, r_fit_m[0], r_fit_m[1])
#     elif r_fit_m[0] < 0:
#         r_rad = -calc_rad(y_eval, r_fit_m[0], r_fit_m[1])
    
#     return l_rad, r_rad

In [ ]:
print('Results: '+str(get_rad_m(data_poly)))

In [ ]:
def merge_pics(img_orig, warped, Minv, rate=0.3):
    size = (warped.shape[1], warped.shape[0]) 
    img_merg = cv2.warpPerspective(warped, Minv, size, flags=cv2.INTER_LINEAR)
    idx = (img_merg == [255,-1,-1])
    idx = np.any(idx, axis=2)
    idx = idx.nonzero()
    # pdb.set_trace()
    out = cv2.addWeighted(img_orig, 1, img_merg, rate, 0)
    out[idx[0],idx[1]] = [255,0,0]
    return out

In [ ]:
res_img = merge_pics(undist_img, out_img, Minv)
f=plot_2pics(out_img, res_img,['Output-Image from Lines Detection','Merged Image'])
plt.savefig('pics/backWarping.png')

In [ ]:
def find_nxt_poly(bi_img, pre_data_poly, margin=100):
    pre_y = pre_data_poly[0]
    l_pre_x = pre_data_poly[1]
    r_pre_x = pre_data_poly[2]
    nonzero = bi_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    l_min = l_pre_x - margin
    l_max = l_pre_x + margin
    r_min = r_pre_x - margin
    r_max = r_pre_x + margin
    
#     pdb.set_trace()
    l_idx = (l_min[nonzeroy] < nonzerox) & (nonzerox < l_max[nonzeroy])
    r_idx = (r_min[nonzeroy] < nonzerox) & (nonzerox < r_max[nonzeroy])
    
    l_y = nonzeroy[l_idx]
    l_x = nonzerox[l_idx]
    r_y = nonzeroy[r_idx]
    r_x = nonzerox[r_idx]
#     l_pts = np.array([nonzeroy[l_idx], nonzerox[l_idx]])
#     r_pts = np.array([nonzeroy[r_idx], nonzerox[r_idx]])
    
    l_fit = np.polyfit(l_y, l_x, 2)
    r_fit = np.polyfit(r_y, r_x, 2)
    
    l_poly_x = calc_poly_x(pre_y, l_fit)
    r_poly_x = calc_poly_x(pre_y, r_fit)
    
    return np.array([pre_y, l_poly_x, r_poly_x])

In [ ]:
f=plot_2pics(img_warped, get_img_out(img_warped,find_nxt_poly(img_warped, data_poly)), ['Binary Image','Detected Lines'],['gray', None])
plt.savefig('pics/linesFinding2.png')

In [ ]:
# global: mtx, dist, M, Minv
# win_w, win_h, margin, min_pix

class Line:
    def __init__(self, data_poly):
        self.dataPoly = deepcopy(data_poly)
#         self.error = None
#     def set_data(self, data_poly):
#         self.dataPoly = data_poly
    def calc_error(self):
        error = self.dataPoly[2]-self.dataPoly[1]
        error = np.max(error)-np.min(error)
        return error
    def smooth_poly(self, line):
#         pdb.set_trace()
        self.dataPoly = (self.dataPoly+line.dataPoly)*0.5
        
class Organizer:
    def __init__(self, errLine, errMatch):
        self.nFailed = 0
        self.preLine = None
        self.errLine = errLine
        self.errMatch = errMatch
        
    def set_pre_line(self, cur_line):
        self.preLine = deepcopy(cur_line)
        
    def is_line(self, line):
#         pdb.set_trace()
        return line.calc_error() < self.errLine
        
    def is_match(self, line):
        if self.preLine is not None:
            diff = np.absolute(np.delete(line.dataPoly,0)-np.delete(self.preLine.dataPoly, 0))
#             pdb.set_trace()
            return np.sum(diff) < self.errMatch
        else:
            return True
    def search_lines_pre_poly(self, bi_img):
        data_poly = find_nxt_poly(bi_img, self.preLine.dataPoly)
        cur_line = Line(data_poly)
        if not (self.is_line(cur_line)):  # not parallel
            self.nFailed += 1                
            debug = True  # for debug
            err_code = 1
        else:  # is a good line
            if not (self.is_match):  # not matching the pre line
                self.nFailed += 1
                debug = True  # for debug
                err_code = 2
            else:  # matched the pre line
                cur_line.smooth_poly(self.preLine)
                self.set_pre_line(cur_line)
                debug = False  # for debug
                self.nFailed = 0
                err_code = -1
        return debug, err_code
    
    def process_img(self, img):
        # get a undistorted img
        undist = cv2.undistort(img, mtx, dist, None, mtx)  
        
        # do img processing, output a bi_img
        bi_s, bi_l_der = convert_hls(undist)
        bi_img = mix_2_chs(bi_s, bi_l_der, False)
        
        # get warped img
        warped = cv2.warpPerspective(bi_img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
        
        nonzero = warped.nonzero()
        # find polynom
        if (self.preLine is None)|(self.nFailed > 5):  # first line or 5 frames failed searching
            data_poly = find_polynom(warped, win_w, win_h, margin)
            cur_line = Line(data_poly)
            if not (self.is_line(cur_line)):  # not parallel
                debug, err_code = self.search_lines_pre_poly(warped)
            else:  # successfully detected
                if self.preLine is not None:
                    cur_line.smooth_poly(self.preLine)
                self.set_pre_line(cur_line)
                self.nFailed = 0
                debug = False  # for debug
                err_code = -1
        else:  # search line using pre_poly
            debug, err_code = self.search_lines_pre_poly(warped)
        if self.preLine is not None:
            data_poly = self.preLine.dataPoly  # get data from pre line
        
            out_img = get_img_out(warped, data_poly)
            res_img = merge_pics(undist, out_img, Minv)
#             rad_l, rad_r= get_rad_m(data_poly)
            rad, offset = get_rad_m(data_poly)
            if (np.absolute(rad)> 4e3)|(rad is None):
                rad_info = "almost straight"
            elif rad > 0:
                rad_info = '{:>6}'.format(int(rad)) + " m to the right"
            elif rad < 0:
                rad_info = '{:>6}'.format(int(-rad))+" m to the left"
                
            if offset > 0:
                offset_info = 'Offset of Car: ' + '{:>6.2f}'.format(offset*100) + ' cm at the right'
            elif offset < 0:
                offset_info = 'Offset of Car: ' + '{:>6.2f}'.format(-offset*100) + ' cm at the left'
            elif offset == 0:
                offset_info = 'Offset of Car: in the middle'
                
            cv2.putText(res_img, "Radius of Curvature: "+rad_info, (10,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0),2)
            cv2.putText(res_img, offset_info, (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0),2)
            
        else:
            res_img = warped
            
            debug = True
            self.nFailed+=1
            err_code = 0
        
        if debug:
            cv2.putText(res_img, "Lines not detected. ErrorCode: "+str(err_code)+ ", nFailed: "+ str(self.nFailed), (10,150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5,(255,0,0),1)
        return res_img

In [ ]:
org = Organizer(100, 1000)
# line = Line(data_poly)
# org.is_line(line)
# org.preLine = deepcopy(line)
# line.dataPoly[1][2] = 0
# org.is_match(line)
f=plot_2pics(raw_img,org.process_img(raw_img),['Original Image','Processed Image'])
plt.savefig('pics/prepVideo.png')

In [ ]:
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML

newpath = 'result'
if not os.path.exists(newpath):
    os.makedirs(newpath)

output = 'result/result.mp4'
clip_orig = VideoFileClip("project_video.mp4")
org = Organizer(120, 1000)

if not os.path.isfile(output):
    res_clip = clip_orig.fl_image(org.process_img) 
    %time res_clip.write_videofile(output, audio=False)
else:
    print('Video Already Exists.')

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>""".format(output))